In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional, Union, Callable
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import logging
from collections import defaultdict
import heapq

In [ ]:

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:

# ==================== ДАТАСЕТЫ ====================

class UniversalRecDataset(Dataset):
    """Универсальный датасет для всех типов задач рекомендаций"""
    def __init__(self, 
                 df: pd.DataFrame,
                 user_col: str = 'user_id',
                 item_col: str = 'item_id', 
                 rating_col: Optional[str] = 'rating',
                 time_col: Optional[str] = 'timestamp',
                 sequence_length: int = 50,
                 task_type: str = 'sequential'):  # 'classic', 'sequential', 'implicit'
        """
        task_type:
        - 'classic': с рейтингами (MSE loss)
        - 'sequential': next-item prediction (NLL loss) 
        - 'implicit': implicit feedback (BPR loss)
        """
        self.df = df.copy()
        self.user_col = user_col
        self.item_col = item_col
        self.rating_col = rating_col
        self.time_col = time_col
        self.sequence_length = sequence_length
        self.task_type = task_type
        
        if task_type == 'sequential':
            # Подготовка для последовательных задач
            self.df = self.df.sort_values([user_col, time_col]).reset_index(drop=True)
            self.user_sequences = self.df.groupby(user_col)[item_col].apply(list).to_dict()
            self.users = list(self.user_sequences.keys())
        else:
            # Для других задач просто сохраняем все взаимодействия
            pass
    
    def __len__(self):
        if self.task_type == 'sequential':
            return len(self.users)
        else:
            return len(self.df)
    
    def __getitem__(self, idx):
        if self.task_type == 'sequential':
            user_id = self.users[idx]
            items = self.user_sequences[user_id]
            
            # Берем последние N взаимодействий
            if len(items) > self.sequence_length:
                sequence = items[-self.sequence_length:]
            else:
                sequence = [items[0]] * (self.sequence_length - len(items)) + items
            
            return {
                'user_id': int(user_id),
                'sequence': torch.LongTensor(sequence[:-1]),
                'target_item': torch.LongTensor([sequence[-1]])
            }
        else:
            row = self.df.iloc[idx]
            result = {
                'user_id': int(row[self.user_col]),
                'item_id': int(row[self.item_col])
            }
            
            if self.rating_col and self.rating_col in row:
                result['rating'] = float(row[self.rating_col])
            
            if self.time_col and self.time_col in row:
                result['timestamp'] = float(row[self.time_col])
                
            return result


In [ ]:

# ==================== МОДЕЛИ ====================

class UniversalRecModel(nn.Module):
    """Универсальная рекомендательная модель"""
    def __init__(self, 
                 n_users: int, 
                 n_items: int,
                 model_type: str = 'mf',  # 'mf', 'gru', 'transformer', 'two_tower'
                 embedding_dim: int = 128,
                 hidden_dim: int = 256,
                 sequence_length: int = 50,
                 n_layers: int = 2):
        super().__init__()
        self.model_type = model_type
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        
        # Базовые эмбеддинги
        self.user_embeddings = nn.Embedding(n_users, embedding_dim)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim)
        
        if model_type == 'mf':
            # Matrix Factorization с bias
            self.user_bias = nn.Embedding(n_users, 1)
            self.item_bias = nn.Embedding(n_items, 1)
            self.global_bias = nn.Parameter(torch.tensor(0.0))
            
        elif model_type == 'gru':
            # GRU для последовательностей
            self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True)
            self.output_projection = nn.Linear(hidden_dim, embedding_dim)
            
        elif model_type == 'transformer':
            # Transformer для последовательностей
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=embedding_dim, 
                nhead=8, 
                dim_feedforward=hidden_dim,
                batch_first=True
            )
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
            self.output_projection = nn.Linear(embedding_dim, embedding_dim)
            
        elif model_type == 'two_tower':
            # Two-tower архитектура
            self.user_projection = nn.Linear(embedding_dim, embedding_dim)
            self.item_projection = nn.Linear(embedding_dim, embedding_dim)
    
    def forward(self, batch: Dict[str, torch.Tensor]) -> torch.Tensor:
        if self.model_type in ['gru', 'transformer']:
            # Для последовательных моделей
            sequences = batch['sequence']  # (batch_size, seq_len)
            embedded_seq = self.item_embeddings(sequences)  # (batch_size, seq_len, embed_dim)
            
            if self.model_type == 'gru':
                gru_out, hidden = self.gru(embedded_seq)
                # Используем последнее скрытое состояние
                user_repr = self.output_projection(hidden[-1])  # (batch_size, embed_dim)
                
            elif self.model_type == 'transformer':
                transformer_out = self.transformer(embedded_seq)
                # Используем последнее состояние
                user_repr = self.output_projection(transformer_out[:, -1, :])  # (batch_size, embed_dim)
            
            # Получаем эмбеддинги целевых айтемов
            target_items = batch['target_item'].squeeze(-1)  # (batch_size,)
            item_repr = self.item_embeddings(target_items)  # (batch_size, embed_dim)
            
            # Скалярное произведение
            scores = torch.sum(user_repr * item_repr, dim=1)
            return scores
            
        elif self.model_type == 'two_tower':
            # Two-tower: отдельные эмбеддинги для юзера и айтема
            user_ids = batch['user_id']
            item_ids = batch['item_id']
            
            user_emb = F.normalize(self.user_embeddings(user_ids), p=2, dim=1)
            item_emb = F.normalize(self.item_embeddings(item_ids), p=2, dim=1)
            
            return torch.sum(user_emb * item_emb, dim=1)
            
        else:  # Matrix Factorization
            user_ids = batch['user_id']
            item_ids = batch['item_id']
            
            user_emb = self.user_embeddings(user_ids)
            item_emb = self.item_embeddings(item_ids)
            
            dot_product = torch.sum(user_emb * item_emb, dim=1)
            
            if hasattr(self, 'user_bias'):
                user_bias = self.user_bias(user_ids).squeeze()
                item_bias = self.item_bias(item_ids).squeeze()
                return dot_product + user_bias + item_bias + self.global_bias
            else:
                return dot_product


In [ ]:

# ==================== МЕТРИКИ ====================

class RecMetrics:
    """Класс для вычисления рекомендательных метрик"""
    
    @staticmethod
    def precision_at_k(y_true: List[int], y_pred: List[int], k: int = 20) -> float:
        """Precision@K"""
        if len(y_pred) > k:
            y_pred = y_pred[:k]
        y_true_set = set(y_true)
        y_pred_set = set(y_pred)
        return len(y_true_set & y_pred_set) / len(y_pred_set) if y_pred_set else 0.0
    
    @staticmethod
    def recall_at_k(y_true: List[int], y_pred: List[int], k: int = 20) -> float:
        """Recall@K"""
        if len(y_pred) > k:
            y_pred = y_pred[:k]
        y_true_set = set(y_true)
        y_pred_set = set(y_pred)
        return len(y_true_set & y_pred_set) / len(y_true_set) if y_true_set else 0.0
    
    @staticmethod
    def map_at_k(y_true: List[int], y_pred: List[int], k: int = 20) -> float:
        """Mean Average Precision@K"""
        if len(y_pred) > k:
            y_pred = y_pred[:k]
        
        score = 0.0
        num_hits = 0.0
        
        for i, p in enumerate(y_pred):
            if p in y_true and p not in y_pred[:i]:
                num_hits += 1.0
                score += num_hits / (i + 1.0)
        
        if not y_true:
            return 0.0
        
        return score / min(len(y_true), k)
    
    @staticmethod
    def ndcg_at_k(y_true: List[int], y_pred: List[int], k: int = 20) -> float:
        """Normalized Discounted Cumulative Gain@K"""
        if len(y_pred) > k:
            y_pred = y_pred[:k]
        
        # Бинарная релевантность
        relevance = [1.0 if item in y_true else 0.0 for item in y_pred]
        
        # DCG
        dcg = sum(rel / np.log2(pos + 2) for pos, rel in enumerate(relevance))
        
        # IDCG
        idcg = sum(1.0 / np.log2(pos + 2) for pos in range(min(len(y_true), k)))
        
        return dcg / idcg if idcg > 0.0 else 0.0


In [ ]:

# ==================== LIGHTNING МОДУЛЬ ====================

class UniversalRecLightning(pl.LightningModule):
    """Универсальный Lightning модуль для всех задач рекомендаций"""
    def __init__(self,
                 model: nn.Module,
                 task_type: str = 'sequential',  # 'classic', 'sequential', 'implicit'
                 learning_rate: float = 1e-3,
                 weight_decay: float = 1e-4,
                 metrics: List[str] = ['map']):
        super().__init__()
        self.model = model
        self.task_type = task_type
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.metrics = metrics
        self.metrics_calculator = RecMetrics()
        
    def forward(self, batch: Dict[str, torch.Tensor]) -> torch.Tensor:
        return self.model(batch)
    
    def training_step(self, batch: Dict[str, torch.Tensor], batch_idx: int) -> torch.Tensor:
        if self.task_type == 'classic':
            # MSE для задач с рейтингами
            predictions = self.model(batch)
            ratings = batch['rating']
            loss = F.mse_loss(predictions, ratings)
            
        elif self.task_type == 'sequential':
            # Cross-entropy для next-item prediction
            logits = self.model(batch)
            target_items = batch['target_item'].squeeze(1)
            loss = F.cross_entropy(logits, target_items)
            
        elif self.task_type == 'implicit':
            # BPR loss для implicit feedback
            scores = self.model(batch)
            loss = -F.logsigmoid(scores).mean()
            
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch: Dict[str, torch.Tensor], batch_idx: int):
        # Вычисление валидационных метрик
        with torch.no_grad():
            predictions = self.model(batch)
            
            if self.task_type == 'classic':
                ratings = batch['rating']
                val_loss = F.mse_loss(predictions, ratings)
                self.log('val_loss', val_loss)
                
            elif self.task_type == 'sequential':
                # Для последовательных задач можно вычислить accuracy
                target_items = batch['target_item'].squeeze(1)
                accuracy = (predictions.argmax(dim=1) == target_items).float().mean()
                self.log('val_acc', accuracy)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay
        )
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min' if self.task_type == 'classic' else 'max',
            factor=0.5,
            patience=3,
            verbose=True
        )
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss' if self.task_type == 'classic' else 'val_acc'
        }


In [ ]:

# ==================== ТРЕЙНЕР ====================

class UniversalRecTrainer:
    """Универсальный трейнер для всех задач рекомендаций"""
    def __init__(self,
                 model_type: str = 'mf',  # 'mf', 'gru', 'transformer', 'two_tower'
                 task_type: str = 'sequential',  # 'classic', 'sequential', 'implicit'
                 embedding_dim: int = 128,
                 hidden_dim: int = 256,
                 sequence_length: int = 50,
                 n_layers: int = 2,
                 learning_rate: float = 1e-3,
                 batch_size: int = 256,
                 max_epochs: int = 50):
        
        self.model_type = model_type
        self.task_type = task_type
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.max_epochs = max_epochs
        
        self.user_encoder = None
        self.item_encoder = None
        self.model = None
        self.lightning_model = None
        self.trainer = None
        
    def prepare_data(self, df: pd.DataFrame, 
                    user_col: str = 'user_id',
                    item_col: str = 'item_id',
                    rating_col: Optional[str] = 'rating',
                    time_col: Optional[str] = 'timestamp',
                    test_size: float = 0.2,
                    val_size: float = 0.1):
        """Подготовка данных с энкодингом и разделением"""
        
        # Создаем энкодеры
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        
        df[user_col] = self.user_encoder.fit_transform(df[user_col])
        df[item_col] = self.item_encoder.fit_transform(df[item_col])
        
        # Разделение данных
        if self.task_type == 'sequential':
            # Для последовательных задач разбиваем по времени
            df = df.sort_values([user_col, time_col])
            # Берем последние interactions как тест
            user_groups = df.groupby(user_col)
            train_data, test_data = [], []
            
            for user_id, user_df in user_groups:
                n_interactions = len(user_df)
                split_idx = int(n_interactions * (1 - test_size))
                train_data.append(user_df.iloc[:split_idx])
                test_data.append(user_df.iloc[split_idx:])
            
            train_df = pd.concat(train_data)
            test_df = pd.concat(test_data)
            
        else:
            # Для других задач случайное разделение
            train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)
        
        # Разделение трейна на трейн и валидацию
        if len(train_df) > len(test_df):  # Убедимся что есть данные для валидации
            train_df, val_df = train_test_split(train_df, test_size=val_size/(1-test_size), random_state=42)
        else:
            val_df = test_df  # Если данных мало, используем тест как валидацию
        
        # Создание датасетов
        train_dataset = UniversalRecDataset(
            train_df, user_col, item_col, rating_col, time_col, 
            self.sequence_length, self.task_type
        )
        
        val_dataset = UniversalRecDataset(
            val_df, user_col, item_col, rating_col, time_col, 
            self.sequence_length, self.task_type
        )
        
        test_dataset = UniversalRecDataset(
            test_df, user_col, item_col, rating_col, time_col, 
            self.sequence_length, self.task_type
        )
        
        return train_dataset, val_dataset, test_dataset
    
    def build_model(self, n_users: int, n_items: int):
        """Создание модели"""
        self.model = UniversalRecModel(
            n_users=n_users,
            n_items=n_items,
            model_type=self.model_type,
            embedding_dim=self.embedding_dim,
            hidden_dim=self.hidden_dim,
            sequence_length=self.sequence_length,
            n_layers=self.n_layers
        )
        
        self.lightning_model = UniversalRecLightning(
            model=self.model,
            task_type=self.task_type,
            learning_rate=self.learning_rate
        )
    
    def train(self, train_dataset, val_dataset, num_workers: int = 4):
        """Обучение модели"""
        
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )
        
        # Callbacks
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss' if self.task_type == 'classic' else 'val_acc',
            mode='min' if self.task_type == 'classic' else 'max',
            save_top_k=1,
            filename='best-{epoch:02d}-{val_loss:.2f}' if self.task_type == 'classic' 
                    else 'best-{epoch:02d}-{val_acc:.2f}'
        )
        
        early_stop_callback = EarlyStopping(
            monitor='val_loss' if self.task_type == 'classic' else 'val_acc',
            patience=10,
            mode='min' if self.task_type == 'classic' else 'max'
        )
        
        # Создание трейнера
        self.trainer = pl.Trainer(
            max_epochs=self.max_epochs,
            callbacks=[checkpoint_callback, early_stop_callback],
            accelerator='auto',
            devices='auto',
            precision=16 if torch.cuda.is_available() else 32,
            gradient_clip_val=1.0
        )
        
        # Обучение
        self.trainer.fit(
            self.lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader
        )
        
        logger.info(f"Обучение завершено. Лучшая модель: {checkpoint_callback.best_model_path}")
    
    def predict_top_k(self, user_ids: List[int], k: int = 20) -> np.ndarray:
        """Генерация top-k рекомендаций для пользователей"""
        self.lightning_model.eval()
        
        # Получаем все возможные айтемы
        all_items = torch.arange(len(self.item_encoder.classes_))
        
        predictions = []
        with torch.no_grad():
            for user_id in user_ids:
                user_tensor = torch.LongTensor([user_id])
                
                if self.model_type in ['gru', 'transformer']:
                    # Для последовательных моделей нужно сгенерировать последовательность
                    # Здесь упрощенный вариант - используем последние взаимодействия
                    # В реальности нужно реализовать proper sequence generation
                    scores = torch.zeros(len(self.item_encoder.classes_))
                else:
                    # Для других моделей
                    user_items_scores = []
                    for item_id in range(len(self.item_encoder.classes_)):
                        batch = {
                            'user_id': user_tensor,
                            'item_id': torch.LongTensor([item_id])
                        }
                        score = self.lightning_model.model(batch)
                        user_items_scores.append(score.item())
                    
                    scores = torch.tensor(user_items_scores)
                
                # Получаем top-k
                top_k = torch.topk(scores, min(k, len(scores)))
                predictions.append(top_k.indices.cpu().numpy())
        
        return np.array(predictions)
    
    def generate_submission(self, test_users: List[int], k: int = 20, 
                           output_file: str = 'submission.csv'):
        """Генерация submission файла"""
        predictions = self.predict_top_k(test_users, k)
        
        # Декодируем ID обратно
        decoded_predictions = []
        for pred in predictions:
            decoded = self.item_encoder.inverse_transform(pred)
            decoded_predictions.append(' '.join(map(str, decoded)))
        
        submission_df = pd.DataFrame({
            'user_id': self.user_encoder.inverse_transform(test_users),
            'item_ids': decoded_predictions
        })
        
        submission_df.to_csv(output_file, index=False)
        logger.info(f"Submission сохранен в {output_file}")


In [ ]:

# ==================== ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ ====================

def example_t_shopping():
    """Пример для T-Shopping задачи (next-item prediction)"""
    # Загрузка данных
    # df = pd.read_parquet('train_data.pq')  # user_id, item_id, date
    
    trainer = UniversalRecTrainer(
        model_type='gru',  # или 'transformer' для лучшего понимания последовательностей
        task_type='sequential',
        embedding_dim=256,  # увеличьте для сложных паттернов
        hidden_dim=512,     # увеличьте для сложных зависимостей
        sequence_length=100, # увеличьте если есть длинные сессии
        learning_rate=1e-3,
        batch_size=128,
        max_epochs=30
    )
    
    # Подготовка данных
    # train_dataset, val_dataset, test_dataset = trainer.prepare_data(
    #     df, user_col='user_id', item_col='item_id', time_col='date'
    # )
    
    # Создание модели
    # trainer.build_model(
    #     n_users=len(trainer.user_encoder.classes_),
    #     n_items=len(trainer.item_encoder.classes_)
    # )
    
    # Обучение
    # trainer.train(train_dataset, val_dataset)
    
    # Генерация предсказаний для submission
    # test_users = list(set(df['user_id']))  # или из sample_submission
    # trainer.generate_submission(test_users, k=20, output_file='t_shopping_submission.csv')

def example_classic_rating():
    """Пример для задач с явными рейтингами"""
    # df = pd.read_csv('ratings.csv')  # user_id, item_id, rating
    
    trainer = UniversalRecTrainer(
        model_type='mf',  # matrix factorization для рейтингов
        task_type='classic',
        embedding_dim=128,
        learning_rate=1e-3,
        max_epochs=50
    )
    
    # Аналогично: prepare_data -> build_model -> train

def example_implicit_feedback():
    """Пример для implicit feedback задач"""
    # df = pd.read_csv('clicks.csv')  # user_id, item_id (без рейтингов)
    
    trainer = UniversalRecTrainer(
        model_type='two_tower',  # good for large-scale implicit
        task_type='implicit',
        embedding_dim=128,
        learning_rate=1e-3
    )
    
    # Аналогично: prepare_data -> build_model -> train


In [ ]:

# ==================== КОНФИГУРАЦИЯ ====================
"""
НАСТРОЙКИ ПОД КОНКРЕТНЫЕ ЗАДАЧИ:

1. T-SHOPPING (Next Item Prediction):
   - model_type: 'gru' или 'transformer'
   - task_type: 'sequential' 
   - sequence_length: 50-200 (в зависимости от сессий)
   - embedding_dim: 256-512
   - Добавить temporal features в preprocessing

2. MOVIELENS (Rating Prediction):
   - model_type: 'mf'
   - task_type: 'classic'
   - loss: MSE
   - embedding_dim: 64-128

3. AMAZON (Implicit Feedback):
   - model_type: 'two_tower'
   - task_type: 'implicit'
   - negative sampling: 10-20 negatives per positive

4. YOUTUBE (Large-scale Retrieval):
   - model_type: 'two_tower' 
   - batch_size: 1024+
   - embedding_dim: 256+
   - use popularity sampling for negatives

ПАРАМЕТРЫ ДЛЯ ТЮНИНГА:
- embedding_dim: 64, 128, 256, 512
- hidden_dim: 128, 256, 512 (для sequential)
- sequence_length: 20, 50, 100 (для sequential)
- learning_rate: 1e-4, 1e-3, 1e-2
- batch_size: 64, 128, 256, 512
- max_epochs: 20-100 (с early stopping)
"""